# Documentação Técnica: Coleta de Dados de Playlists do Spotify e Exportação para Google Sheets

## Descrição Geral:
Este script Python tem como objetivo buscar informações sobre as músicas presentes nas playlists de "Top 50 Brasil" para cada ano de uma lista específica de décadas, utilizando a biblioteca Spotipy para acessar a API do Spotify. Além disso, ele exporta os dados coletados para uma planilha do Google Sheets.

## Dependências Externas:
- `pprint`: Biblioteca Python para "pretty-printing" estruturas de dados complexas, usada para imprimir resultados de forma legível.
- `pandas`: Biblioteca Python para manipulação e análise de dados, utilizada para criar e manipular estruturas de dados tabulares.
- `spotipy`: Biblioteca Python que serve como um cliente para a API Web do Spotify, permitindo acesso a diversos recursos da plataforma Spotify.
- `gspread`: Biblioteca Python para interagir com o Google Sheets.
- `oauth2client`: Biblioteca Python para lidar com a autenticação OAuth 2.0.

## Variáveis de Ambiente:
- `client_id`: Identificador único do cliente do aplicativo fornecido pelo Spotify para autenticação.
- `client_secret`: Chave secreta do cliente do aplicativo fornecida pelo Spotify para autenticação.
- `decadas`: Lista contendo os anos específicos de cada década que serão utilizados para buscar as playlists de "Top 50 Brasil".
- `id`: ID da playlist obtido através da busca na API do Spotify.

## Fluxo de Execução:
1. O script importa as bibliotecas necessárias: `pprint`, `pandas`, `spotipy`, `gspread`, e `ServiceAccountCredentials` do `oauth2client`.
2. Ele inicializa a instância `SpotifyClientCredentials` com as credenciais do cliente (ID do cliente e chave secreta).
3. Cria uma instância do cliente do Spotify (`sp`) utilizando o gerenciador de credenciais do Spotify.
4. Itera sobre cada ano especificado na lista `decadas`.
5. Para cada ano, busca a playlist de "Top 50 Brasil" correspondente através da função `sp.search`.
6. Extrai os dados relevantes de cada faixa da playlist, como nome da música, nome do artista, tempo, álbum, data de lançamento e gênero musical.
7. Armazena os dados de cada faixa em um dicionário `info_faixa`.
8. Adiciona os dados da faixa ao `Lista_total` e o ano correspondente à `anos_consulta`.
9. Cria um DataFrame do pandas (`df_playlist`) a partir dos dados coletados.
10. Exibe o DataFrame resultante na saída padrão e imprime sua forma (shape).
11. Configura as credenciais do serviço Google Sheets usando o arquivo JSON fornecido pelo Google Cloud Platform.
12. Abre a planilha do Google Sheets especificada pelo ID fornecido.
13. Exporta o DataFrame `df_playlist` para a planilha do Google Sheets.

## Saída:
A saída do script é um DataFrame do pandas contendo informações detalhadas sobre as músicas coletadas das playlists de "Top 50 Brasil" para cada ano de uma lista específica de décadas. Além disso, os dados são exportados para uma planilha do Google Sheets especificada.

## Observações:
- Certifique-se de substituir `client_id` e `client_secret` com as credenciais do seu próprio aplicativo Spotify.
- A lista `decadas` deve conter os anos específicos de cada década que você deseja consultar.
- Substitua o nome do arquivo JSON contendo suas credenciais do Google Sheets e o ID da planilha do Google Sheets que deseja usar.
- Este script requer autenticação com credenciais válidas do Spotify e do Google Sheets para acessar as informações das playlists e exportar os dados para a planilha.

In [ ]:
from pprint import pprint
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
client_id = 'app_client'
client_secret = 'app_sct'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
decadas=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
Lista_total =[]
anos_consulta =[]

for ano in decadas:    
    ano_especifico = ano  # Substitua pelo ano desejado

# Realize a busca pela playlist de top músicas do Brasil em um ano específico
    busca_playlist = sp.search(q=f'top 50 Brazil {ano_especifico}', type='playlist', limit=1)

    id = busca_playlist['playlists']['items'][0]['id']
    playlist = sp.playlist_tracks(id)


    for item in playlist['items']:
        faixa = item['track']
        tempo_ms = faixa['duration_ms']

        # Converter milissegundos para minutos e segundos
        minutos = tempo_ms // 60000  # 1 minuto = 60000 milissegundos
        segundos = (tempo_ms % 60000) // 1000  # O restante são os segundos

        # Formatando o tempo em minutos e segundos juntos (ex: '4:30')
        tempo_formatado = f'{minutos}:{segundos:02}'  

        artistas = faixa['artists']
        generos = []

        for artista in artistas:
            # Pesquisar informações sobre o artista
            artista_info = sp.artist(artista['id'])
            generos.extend(artista_info['genres'])
        genero_musical = generos[0] if generos else 'Desconhecido'

        info_faixa = {
            'nome_da_muc': faixa['name'],
            'Artista': artistas[0]['name'],
            'tempo': tempo_formatado,
            'album': faixa['album']['name'],
            'Data_de_lançamento': faixa['album']['release_date'],
            'Gênero': genero_musical
            
        }

        Lista_total.append(info_faixa)
        anos_consulta.append(ano)

df_playlist = pd.DataFrame(Lista_total)
df_playlist['Ano'] = anos_consulta
display(df_playlist)
print(df_playlist.shape)

In [ ]:
#ESTE BLOCO DE CÓDIGO GARAVA TODO O DATAFRAME DE UMA VEZ 
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('arquivo com suas credenciais.json', scope)
gc = gspread.authorize(credentials)

wks = gc.open_by_key('coloque aqui o ID da sua planilha')
worksheet = wks.get_worksheet(0)
from gspread_dataframe import get_as_dataframe, set_with_dataframe
set_with_dataframe(worksheet, df_playlist)
